In [1]:
from keras.layers import Input, Dense, BatchNormalization, Activation
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, Concatenate
from keras.layers import LeakyReLU, Dropout, Lambda, Activation, BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
n_mels= 128
n_frames= 64
n_hop_frames= 8
n_fft= 1024
hop_length=512
power= 2.0
n_conditions = 3
lr = 0.001

def get_model(n_frames, n_mels, n_conditions, latent_dim=100):
    in_image = Input(shape= (n_frames, n_mels, 3))
    X = Conv2D(32, (3,3), strides=(2,2), padding='same')(in_image)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(64, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(128, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Flatten()(X)
    X = Dropout(0.4)(X) #Consider adding more dropout layers to minimize overfitting - remember we work with limited labeled data. 
    X = Dense(latent_dim)(X)
    
    model = Model(inputs=in_image, outputs=X)
    
    return model

enc_model= get_model(n_frames = n_frames, n_mels = n_mels, n_conditions = n_conditions)
# print(enc_model.summary())

def decoder(n_conditions, latent_dim = 100):
	
	in_lat = Input(shape=(latent_dim,))
	#Start with enough dense nodes to be reshaped and ConvTransposed to 28x28x1
	n_nodes = 256 * 16 * 32
	X = Dense(n_nodes)(in_lat)
	X = LeakyReLU(alpha=0.2)(X)
	X = Reshape((16, 32, 256))(X)
	
	X = Conv2DTranspose(128, (3,3), strides=(2,2), padding='same')(X) #14x14x128
	X = LeakyReLU(alpha=0.2)(X)
	
	X = Conv2DTranspose(64, (3,3), strides=(1,1), padding='same')(X) #14x14x64
	X = LeakyReLU(alpha=0.2)(X)
	# output
	out_layer = Conv2DTranspose(n_conditions, (3,3), strides=(2,2), activation='tanh', 
                             padding='same')(X) #28x28x1
	# define model
	model = Model(in_lat, out_layer)
	return model

dec_model=decoder(n_conditions = n_conditions)
# print(dec_model.summary())

def define_generator(enc_model, dec_model):
    # dec_model.trainable = False
    gen_output = dec_model(enc_model.output) #Gen. output is the input to disc. 
    model = Model(enc_model.input, gen_output)
    # model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))
    return model

gen_model = define_generator(enc_model, dec_model)
# print(gen_model.summary())

def define_discriminator(n_frames, n_mels, n_classes= n_conditions):
    in_image = Input(shape=(n_frames, n_mels, 3))
    X = Conv2D(32, (3,3), strides=(2,2), padding='same')(in_image)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(64, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Conv2D(128, (3,3), strides=(2,2), padding='same')(X)
    X = LeakyReLU(alpha=0.2)(X)
    
    X = Flatten()(X)
    X = Dropout(0.4)(X) #Consider adding more dropout layers to minimize overfitting - remember we work with limited labeled data. 
    X = Dense(n_classes)(X)
    
    model = Model(inputs=in_image, outputs=X)
    
    return model

disc_model=define_discriminator(n_frames = n_frames, n_mels = n_mels)
# print(disc_model.summary())

def define_gan(gen_model, disc_model):
	
    disc_model.trainable = False # make unsup. discriminator not trainable
    gan_output = disc_model(gen_model.output) #Gen. output is the input to disc. 
    model = Model(gen_model.input, gan_output)
    model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001, beta_1=0.5))
    return model

gan_model = define_gan(gen_model, disc_model)
print(gan_model.summary())

    